In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
medical_df=pd.read_csv('/kaggle/input/medical/insurance.csv')

In [ ]:
medical_df.head()

In [ ]:
medical_df.info()

The dataset contains 1338 rows and 7 columns.
our objective is to find a way to estimate value in the charges column using the values from the other columns.
we should be able to estimate charges for new customers too just by asking for information like sex,age,bmi,etc
Good thing is that data doesnot contains any missing values so no need to clean

In [ ]:
medical_df.describe()

In the charges column we can understand that there is huge variation in the charges from 1k to 64k. And we could also understand that there is huge skew/several outliers. We  infernce this as the 50%(median value ) is much lower than the maximun charges

In [ ]:
import plotly.express as px
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

# AGE

Age is numeric data.the minimum age is 18 years and maximum is 64 years. thus, we can visualize the distribution by (64-18+1)=47 bins i.e one for each year 

In [ ]:
medical_df.age.describe()

In [ ]:
fig= px.histogram(medical_df,
                 x='age',
                 marginal='box',
                 nbins=48,
                 title='Distribution of age ')
fig.update_layout(bargap=0.2)
fig.show()

The distribution is almost uniform, with 20-30 customers per age group,except the ages 18-19,which seems to have over twice the customers

# BMI

In [ ]:
fig= px.histogram(medical_df,
                 x='bmi',
                 marginal='box',
             color_discrete_sequence=['green'],
                 title='Distribution of bmi ')
fig.update_layout(bargap=0.2)
fig.show()

the measurements of bmi seems to form a 'Gussian distribution' centered around the 30 value

# CHARGES

In [ ]:
fig= px.histogram(medical_df,
                 x='charges',
                 marginal='box',
                  color='smoker',
             color_discrete_sequence=['red','grey'],
                 title='Distribution of Charges')
fig.update_layout(bargap=0.2)
fig.show()

For mst of the customers,the annual charges are under 10k. Only a small fraction of customer have higher medical expenses,possibly due to accidents or major illiness.
There is considerable difference between the medical expenses between a smokerr and non smoker. the median for the non smoker is around 7 whereas for the smoker is around $35

# REGION

In [ ]:
fig= px.histogram(medical_df,
                 x='region',
                 marginal='box',
                  color='smoker',
             color_discrete_sequence=['yellow','red'],
                 title='Distribution of bmi ')
fig.update_layout(bargap=0.2)
fig.show()

# SMOKER

In [ ]:
fig= px.histogram(medical_df,
                 x='smoker',
                  color='sex',
             color_discrete_sequence=['grey','blue'],
                 title='Distribution of bmi ')

fig.show()

Says that smoking is more common among males.

# Age and Charges

In [ ]:
fig= px.scatter(medical_df,
                 x='age',
                y='charges',
                color='smoker',
                 opacity=0.8,
             hover_data=['sex'],
                 title='Age vs Charges')
fig.update_traces(marker_size=5)
fig.show()

we can observe that 
* the medical expenses increases as the age increrases,however we see various variation at every age so age alone cannot be the only factor,
* the first and largest cluster, which seems to follow a linear pattern shows that non smokers have presumbly a healthier lifestyle as the medical expenses verify that.
* the second cluster contains a mix of smokers and non smokers. It is possible that it contains two overlapping clusters of "smokers without major health issues" and " nono smokers with medical issues".
* third cluster shows the smokers with major medical issues, possibly worsend due to smoking

# Bmi vs Charges

In [ ]:
fig= px.scatter(medical_df,
                 x='bmi',
                y='charges',
                color='smoker',
                 opacity=0.8,
             hover_data=['sex'],
                 title='BMI vs Charges')
fig.update_traces(marker_size=5)
fig.show()

Seems like increased BMI for the non smokers did not account for more medical expenses but for smokers with a BMI greater than 30 showed increased medical expenses

# Correlation

In [ ]:
smoker_values={'no':0,'yes':1}
smoker_numerical=medical_df.smoker.map(smoker_values)
smoker_numerical

.corr method is used to compare the relationship between columns and their correlation, we need numeric data to do so, therefore we convert smoker string data into numeric 

In [ ]:
medical_df.head()

we exclude the data type :objects to get the correlation matrix

In [ ]:
medical_without_categoricals = medical_df.select_dtypes(exclude=['object'])
medical_without_categoricals.corr()

# Linear regression using single feature

we know that the smoker and age column have the strongest correlation with the charges column.
Let's find a way to estimate charges using the value of age for non smokers

In [ ]:
non_smoker_df=medical_df[medical_df.smoker=='no']
plt.title('Age vs Charges')
sns.scatterplot(data=non_smoker_df,x='age',y='charges',alpha=0.7,s=15);

we'll try and fit a line using this points 
y=wa+b
* **MODEL**
* charges=W X Age + b...... w is called weight and b is called the bias
* this technique is called linear regression, because it models the relationship between age and charges as a straight line
* the values in the age column of the dataset is called the inputs to the model and the values in the charges column is called the targets

In [ ]:
def estimate_charges(age,w,b):
    return w*age+b

We defined a helper function estimate_charges.
this is my first machine learning model
Lets guess the value of w and b and use them to estimate the value of charges

In [ ]:
w=50
b=100

In [ ]:
estimate_charges(45.66,w,b)

In [ ]:
ages=non_smoker_df.age
ages


In [ ]:
estimated_charges=estimate_charges(ages,w,b)
estimated_charges

In [ ]:
non_smoker_df.charges

By comparing we see that the estimated charges are off by a great numbers....pretty baddd

In [ ]:
target=non_smoker_df.charges
plt.plot(ages,estimated_charges,'r',alpha=0.9);
plt.scatter(ages,target,s=8,alpha=0.8);
plt.xlabel('Age');
plt.ylabel('Charges');
plt.legend(['Estimate','Actual']);

**clearly we can see that our estimates are quite poor and the line doesnot fit**
let's create a helper function to try diff values of  w and b

In [ ]:
def try_parameters(w,b):
    ages=non_smoker_df.age
    target=non_smoker_df.charges
    estimated_charges=estimate_charges(ages,w,b)
plt.plot(ages,estimated_charges,'r',alpha=0.9);
plt.scatter(ages,target,s=8,alpha=0.8);
plt.xlabel('Age');
plt.ylabel('Charges');
plt.legend(['Estimate','Actual']);

In [ ]:
try_parameters(500,-700)

# LOW/COST FUNCTION
we can compare our model's prediction with the actual targets using this:
* calculate the difference between the targets and predictions(the diff is called the residual)
* square all elements of the diff to remove negative values 
* calculate the average of the elements in result
* take the square root of the results
****the result is known as RSME(root mean sqaured error)****


In [ ]:
def rmse(targets,predictions):
    return np.sqrt(np.mean(np.square(targets - predictions)))

In [ ]:
w=299
b=-4700

In [ ]:
targets=non_smoker_df['charges']
predictions=estimate_charges(non_smoker_df.age,w,b)

In [ ]:
rmse(targets,predictions)

****On average,each element in the prediction differs from the actual targets by $8641**
$which is pretty badd and result is called loss**

In [ ]:
def try_parameters(w,b):
    ages=non_smoker_df.age
    target=non_smoker_df.charges
    estimated_charges=estimate_charges(ages,w,b)
plt.plot(ages,estimated_charges,'r',alpha=0.9);
plt.scatter(ages,target,s=8,alpha=0.8);
plt.xlabel('Age');
plt.ylabel('Charges');
plt.legend(['Estimate','Actual']);
print(rmse)

# scikit-learn
in daily practice we will never use the above used methods......that was all for the understandingg... yehahh damn shit anyways... 


In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model=LinearRegression()

In [ ]:
help(model.fit)

In [ ]:
inputs=non_smoker_df[['age']]
targets=non_smoker_df.charges
print('inputs.shape and target.shape',inputs.shape,targets.shape)

model.fit requires two dimensions therefore we do [['age']] we want a dataframe not series
the targets can be a array

In [ ]:
model.fit(inputs,targets)

In [ ]:
model.predict(np.array([[23],[35],[62]]))

looks reasonable to me tho, lets try computing for the whole set of inputs

In [ ]:
predictions=model.predict(inputs)

In [ ]:
predictions

In [ ]:
targets

**pretty acceptable******

# lets compute the rsme loss 

In [ ]:
rmse(targets,predictions)

# our prediction is offf by somewhat 4662 which is not bad considering the fact that there are several outliers

In [ ]:
#value of w using sk lear
model.coef_

In [ ]:
#value of b using sk lear
model.intercept_

# MACHINE LEARNING IS JUST GLORIFIED LINE FITTING

# **# It just takes a few lines of code to train a machine learning model using scikit learn**

In [ ]:
#create inputs and targets
inputs,targets= non_smoker_df[['age']],non_smoker_df['charges']

#create and train the model
model=LinearRegression().fit(inputs,targets)

#generate the predicitons
predictions=model.predict(inputs)

#compute loss to evalute the model
loss=rmse(targets,predictions)
print('loss:',loss)


# LINEAR REGRESSION USING MULTIPLE FEATURES 
so far, We've only used 'age' feature to estimate 'charges'. adding another feature like 'bmi' is fairly straightforward
* CHARGES= W1Xage+W2Xbmi+b
* meaning we just need to change one line of the above code

In [ ]:
#create inputs and targets
inputs,targets= non_smoker_df[['age','children','bmi']],non_smoker_df['charges']

#create and train the model
model=LinearRegression().fit(inputs,targets)

#generate the predicitons
predictions=model.predict(inputs)

#compute loss to evalute the model
loss=rmse(targets,predictions)
print('loss:',loss)


**BMI doesn't seems to reduce the loss by much,as the BMI has a very less correlation**

In [ ]:
#create inputs and targets
inputs,targets= medical_df[['age','children','bmi','sex_codes']],medical_df['charges']

#create and train the model
model=LinearRegression().fit(inputs,targets)

#generate the predicitons
predictions=model.predict(inputs)

#compute loss to evalute the model
loss=rmse(targets,predictions)
print('loss:',loss)


In [ ]:
model.predict([[29,0,30,1]])

In [ ]:
model.coef_

# use of Categorial data for ML
1. if categorial column just has two categories,then we can replace their value with 0 and 1
1. if more than 2 column, we can perform one-hot encoding i.e create a new column for each category with 1s and 0s
1. if the categories have a natural order(eg. cold neutral,warm,hot),then they can be convertd to numbers(eg.1,2,3,4)

Lets try adding sex column

# MODEL IMPROVEMENT
scaling of features using scikit learn

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
numeric_cols=['age','bmi','children']
scaler=StandardScaler()
scaler.fit(medical_df[numeric_cols])

In [ ]:
scaled_inputs=scaler.transform(medical_df[numeric_cols])
scaled_inputs

CREATING A TEST SET-


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
inputs_train,inputs_test,targets_train,targets_test=train_test_split(inputs,targets,test_size=0.1)

In [ ]:
inputs_test
